### Table comparing barcoded Prom and Gridion
Generates comparison tables for unblocked and sequenced metrics for the two tables.
Goes from sequnecing summary files for these experiments, which are trimmed to only keep the columns we need, and are sorted based on the presence of the read_id in the `unblocked_read_ids.txt` from readfish.

Note for me:
fast5s for gridion run are here:
`/mnt/wastewater/scratch_backup/ML_Runs_GridION/AP_002/BC-3-Panel-Human/20211025_1338_X1_FAQ39590_af835873/fast5*`

fast5s for prom run are here:
`/mnt/waterprom/ml_runs/PAS_001/NA12878_05_NB4_06_22Rv1_07`

Separated with `rftools` which requires the unblocked read ids file, https://github.com/LooseLab/rftools

```bash
rftools split-ss -p <PREFIX> `20220317_1702_3A_PAK09329_4cae256c` sequencing_summary_PAK09329_e742ec3b.txt
```

where <PREFIX> is one of a combination of sequencing device and run number, i.e grid1, prom2 etc.

Split sequencing summarys were trimmed to only contain relevant information using `qsvlite` - see https://github.com/jqnatividad/qsv?tab=readme-ov-file#option-3-install-with-rust
Using the select command:
```bash
fd -e.txt ed -x qsvlite select -d "\t" -o {.}_lite.tsv read_id,sample_id,sequence_length_template,barcode_arrangement {}
```

And then compressed with `xz` (in parallel with `fd`, just repalce {} with the filename to be compressed if not using `fd`

```bash
fd -tf lite -x xz -T 24 -e -z {}
```

### Alignment
Alignment was performed using `minimap2 2.26-r1175`
A script for alignment (`align_barcode_fastq`) is provided in the table 1 directory of the repository, the `map-ont` preset is used. This runs the alignment in Parallel for all files using 16 cores, so mayeb edit that.
Both PASS and FAIL data is included.
The data was called with HAC.

### For coverage on and off target
To generate files for on and off target coverage we used `mosdepth 0.3.6`.
First the reads were mapped to Hg38.p14 with all alt and unplaced contigs removed, leaving only the main chromosome assemblies. 
Inverse complements regions (for off target based coverage) to the target bed files used were generated with `bedtools v2.31.1`
This requires the bedfile to be sorted, this can be done with `bedtools sort`.

```bash
fd -e .sorted.bed -x sh -c 'bedtools complement -i -g /data/refs/hg38_simple.genome > {.}_inverse_complement.bed'
```
These are located in the target_beds.

Mosdepth was used to calculate median coverage over the regions, and the median of this coverage was taken in this notebook. The targets were correct for each barcode. And example command:
```bash
mosdepth -m -t 32 -b CancerPanelTargets.sorted.bed -xn all_barcode_07_sequenced ../merged_reads/all_barcode07.sequenced.fastq.bam
```

An example script to run this in parallel for the Gridion run data is provided, `mosdepth_bam_files`. Again in parallel so user beware thread counts.

##### Unclassified coverage
For unclassified on/off target coverage, all three gene panel target bed files were merged, and overlaps were collapsed, but with strand awareness for the merging.
Merging was performed with `bedtools v2.31.1` using the following command - 
```bash
fd -ae sorted.bed -X cat {} | save -f all_targets.bed
bedtools sort -i all_targets.bed | save -f all_targets.sorted.bed
bedtools merge -s -i all_targets.sorted.bed | save -f all_targets.merged.sorted.bed
bedtools complement -i all_targets.merged.sorted.bed -g /data/refs/hg38_simple.genome | save -f all_targets.sorted_inverse_complement.bed
```

The script for mosdepth also does the coverage for on/off target coverage in unclassified.

## Warning
This notebook is memory hungry (>20Gb RAM required). If this is an issue, contact us and we will work out a way to calculate these metrics without loading everything into a dataframe.

In [175]:
import gc
import re
import sys
from itertools import chain, repeat, zip_longest
from pathlib import Path

import numpy as np
import pandas as pd

PAT = re.compile(r"(sequenced|unblocked)")
gm, nb, rv = (
    ["GM12878", "TruSight 170 Tumour Panel", 170],
    ["NB4", "TruSight RNA Fusion Panel", 508],
    ["22Rv1", "COSMIC", 717],
)
gridion_samples = {
    "barcode01": gm,
    "barcode02": nb,
    "barcode03": rv,
    "unclassified_grid": ["*", "*", "*"],
}
prom_samples = {
    "barcode05": gm,
    "barcode06": nb,
    "barcode07": rv,
    "unclassified_prom": ["*", "*", "*"],
}

In [2]:
def n50(lengths: pd.Series) -> float:
    """
    Returns the N50 of a series of read lengths
    """
    lengths = lengths.sort_values(ascending=False).values
    cumulative_sum = lengths.cumsum()
    total_length = cumulative_sum[-1]
    n50_length = lengths[(cumulative_sum >= total_length / 2).argmax()]
    return n50_length

In [3]:
dfs = []
for summary in Path(".").rglob("gridion*.tsv.xz"):
    read_type = PAT.findall(summary.name)[0]
    df = pd.read_csv(summary, sep="\t")
    df["read_type"] = read_type
    dfs.append(df)

gridion_df = pd.concat(dfs)

In [4]:
gridion_summary = gridion_df.groupby(["read_type", "barcode_arrangement"])[
    "sequence_length_template"
].agg([np.mean, np.sum, n50])

/tmp/ipykernel_684477/1983675264.py:3: FutureWarning: The provided callable <function mean at 0x7fcd4c09c700> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ].agg([np.mean, np.sum, n50])
/tmp/ipykernel_684477/1983675264.py:3: FutureWarning: The provided callable <function sum at 0x7fcd4c09b250> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ].agg([np.mean, np.sum, n50])


In [5]:
gridion_summary = gridion_summary.loc[
    :, ["barcode01", "barcode02", "barcode03", "unclassified"], :
]

## Promethion

In [6]:
dfs = []
for summary in Path(".").rglob("sequencing_summaries/prom*.tsv.xz"):
    read_type = PAT.findall(summary.name)[0]
    df = pd.read_csv(summary, sep="\t")
    df["read_type"] = read_type
    dfs.append(df)

prom_df = pd.concat(dfs)

In [7]:
prom_summary = prom_df.groupby(["read_type", "barcode_arrangement"])[
    "sequence_length_template"
].agg([np.mean, np.sum, n50])

/tmp/ipykernel_684477/4195199970.py:3: FutureWarning: The provided callable <function mean at 0x7fcd4c09c700> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ].agg([np.mean, np.sum, n50])
/tmp/ipykernel_684477/4195199970.py:3: FutureWarning: The provided callable <function sum at 0x7fcd4c09b250> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ].agg([np.mean, np.sum, n50])


In [8]:
prom_summary = prom_summary.loc[
    :, ["barcode05", "barcode06", "barcode07", "unclassified"], :
]

In [9]:
gridion_summary = gridion_summary.reset_index()
gridion_summary["barcode_arrangement"] = gridion_summary["barcode_arrangement"].map(
    lambda x: x if x != "unclassified" else "unclassified_grid"
)
gridion_summary["Device"] = "G."

In [10]:
prom_summary = prom_summary.reset_index()
prom_summary["barcode_arrangement"] = prom_summary["barcode_arrangement"].map(
    lambda x: x if x != "unclassified" else "unclassified_prom"
)
prom_summary["Device"] = "P."

In [11]:
testing = pd.concat((gridion_summary, prom_summary))

In [12]:
testing = testing.rename(
    columns={
        "barcode_arrangement": "Barcode",
        "mean": "Mean",
        "sum": "Yield",
        "n50": "N50",
    }
)
testing["read_type"] = testing["read_type"].map(
    {"sequenced": "Seq.", "unblocked": "Unb."}
)

In [116]:
t = testing.pivot(
    index=["Barcode", "Device"],
    columns="read_type",
    values=[
        "Yield",
        "Mean",
        "N50",
    ],
)

In [117]:
prom_samples.update(gridion_samples)

In [118]:
t[["Sample", "Panel", "Gene number"]] = pd.DataFrame.from_records(
    t.index.get_level_values(0).map(prom_samples), index=t.index
)

#### Get the regions from the mosdepth files


In [119]:
def read_coverages_of_regions() -> pd.DataFrame:
    """Read the mosdepth coverage region bed files filtering to the given barcodes.
    Create a concatenated datafrom of all files, with barcode, coverage and on and off target and return it.
    """
    prom = ("prom", "05", "06", "07", "unclassified_prom")
    grid = ("grid", "01", "02", "03", "unclassified_grid")
    pat = re.compile(r"(barcode|unclassified[_promgrid]{5})_*(\d{2})*.*")
    coverages = []
    for barcode, device in chain(
        zip_longest(prom[1:], [], fillvalue=prom[0]),
        zip_longest(grid[1:], [], fillvalue=grid[0]),
    ):
        files = list(Path("mosdepth_region_summaries/").glob(f"*{barcode}*.bed.gz"))
        # print(files)
        for region_file in files:
            region_type = (
                "Off target"
                if "off_target" in str(region_file) or "inverse" in str(region_file)
                else "On target"
            )
            print(f"\t{region_file}\t{region_type}")
            barcode = "".join(pat.findall(str(region_file))[0])
            # Unclassified and off target bed files do not provide names for records so we shift index for Coverage column down one
            usecols = (
                [3]
                if region_type == "Off target" or "unclassified" in str(region_file)
                else [4]
            )
            df = pd.read_csv(
                region_file, sep="\t", header=None, usecols=usecols, names=["Coverage"]
            )
            df["barcode"] = barcode
            df["region_type"] = region_type
            df["Device"] = f"{device[0].upper()}."
            coverages.append(df)
    return pd.concat(coverages)

In [120]:
coverage_df = read_coverages_of_regions()

	mosdepth_region_summaries/all_barcode_05.regions.bed.gz	On target
	mosdepth_region_summaries/all_barcode_05_off_target.regions.bed.gz	Off target
	mosdepth_region_summaries/all_barcode_06.regions.bed.gz	On target
	mosdepth_region_summaries/all_barcode_06_off_target.regions.bed.gz	Off target
	mosdepth_region_summaries/all_barcode_07.regions.bed.gz	On target
	mosdepth_region_summaries/all_barcode_07_off_target.regions.bed.gz	Off target
	mosdepth_region_summaries/unclassified_prom_off_target.regions.bed.gz	Off target
	mosdepth_region_summaries/unclassified_prom_on.regions.bed.gz	On target
	mosdepth_region_summaries/barcode01_tst_170_coords_extended.sorted.regions.bed.gz	On target
	mosdepth_region_summaries/barcode01_tst_170_coords_extended.sorted_inverse_complement.regions.bed.gz	Off target
	mosdepth_region_summaries/barcode02_tst_fusion_coords_extended.sorted_inverse_complement.regions.bed.gz	Off target
	mosdepth_region_summaries/barcode02_tst_fusion_coords_extended.sorted.regions.bed.gz

In [121]:
a = (
    coverage_df.groupby(["barcode", "region_type", "Device"])["Coverage"]
    .median()
    .reset_index()
    .pivot(index=["barcode", "Device"], columns="region_type", values="Coverage")
)

In [122]:
a.index.names = ["Barcode", "Device"]

In [123]:
a.dtypes

region_type
Off target    float64
On target     float64
dtype: object

In [124]:
t[["Median Off Target Coverage", "Median Target Coverage"]] = a

In [125]:
t = t.reindex(
    [
        ("barcode01", "G."),
        ("barcode02", "G."),
        ("barcode03", "G."),
        ("unclassified_grid", "G."),
        ("barcode05", "P."),
        ("barcode06", "P."),
        ("barcode07", "P."),
        ("unclassified_prom", "P."),
    ]
)

In [126]:
t

Yield                       Mean              \
read_type                         Seq.          Unb.         Seq.        Unb.   
Barcode           Device                                                        
barcode01         G.      3.345244e+08  3.465450e+09  1926.417947  554.378839   
barcode02         G.      1.238984e+09  4.839475e+09  4203.349585  550.872480   
barcode03         G.      1.248408e+09  3.835499e+09  5065.192309  555.972116   
unclassified_grid G.      1.709108e+08  3.658782e+09   798.171085  733.417091   
barcode05         P.      1.278234e+09  1.476375e+10   930.660416  897.375697   
barcode06         P.      4.357606e+09  2.339281e+10  1914.922736  903.235613   
barcode07         P.      3.010605e+09  1.363278e+10  2224.730111  909.816885   
unclassified_prom P.      7.036909e+08  1.549620e+10   507.391720  962.700204   

                             N50          Sample                      Panel  \
read_type                   Seq.   Unb.                                       
Barcode           Device                                                      
barcode01         G.      8149.0  555.0  GM12878  TruSight 170 Tumour Panel   
barcode02         G.      7191.0  552.0      NB4  TruSight RNA Fusion Panel   
barcode03         G.      6858.0  556.0    22Rv1                     COSMIC   
unclassified_grid G.       923.0  792.0        *                          *   
barcode05         P.      7163.0  917.0  GM12878  TruSight 170 Tumour Panel   
barcode06         P.      7349.0  919.0      NB4  TruSight RNA Fusion Panel   
barcode07         P.      6999.0  923.0    22Rv1                     COSMIC   
unclassified_prom P.       543.0  989.0        *                          *   

                         Gene number Median Off Target Coverage  \
read_type                                                         
Barcode           Device                                          
barcode01         G.             170                      1.000   
barcode02         G.             508                      1.000   
barcode03         G.             717                      1.000   
unclassified_grid G.               *                      0.000   
barcode05         P.             170                      4.000   
barcode06         P.             508                      7.635   
barcode07         P.             717                      4.000   
unclassified_prom P.               *                      4.000   

                         Median Target Coverage  
read_type                                        
Barcode           Device                         
barcode01         G.                      12.00  
barcode02         G.                      15.00  
barcode03         G.                      12.00  
unclassified_grid G.                       1.00  
barcode05         P.                      35.00  
barcode06         P.                      52.00  
barcode07         P.                      27.25  
unclassified_prom P.                       5.00

In [127]:
t["Fold Enrichment"] = t["Median Target Coverage"] / t["Median Off Target Coverage"]

In [128]:
t.loc[["unclassified_grid", "unclassified_prom"], "Fold Enrichment"] = "*"

/tmp/ipykernel_684477/438197521.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '*' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  t.loc[["unclassified_grid", "unclassified_prom"], "Fold Enrichment"] = "*"


In [129]:
t.index

MultiIndex([(        'barcode01', 'G.'),
            (        'barcode02', 'G.'),
            (        'barcode03', 'G.'),
            ('unclassified_grid', 'G.'),
            (        'barcode05', 'P.'),
            (        'barcode06', 'P.'),
            (        'barcode07', 'P.'),
            ('unclassified_prom', 'P.')],
           names=['Barcode', 'Device'])

In [130]:
 total_df= t.groupby(level=1).agg({('Mean', 'Seq.'): "mean", ('Mean', 'Unb.'): "mean", ('Yield', 'Seq.'): "sum", ('Yield', 'Unb.'): "sum"}).reset_index()
 total_df["Barcode"] = "Total"
 t = pd.concat((t, total_df.set_index(["Barcode", "Device"])))
 t = t.reindex([('barcode01', 'G.'), ('barcode02', 'G.'), ('barcode03', 'G.'),
       ('unclassified_grid', 'G.'), ('Total', 'G.'), ('barcode05', 'P.'),
       ('barcode06', 'P.'), ('barcode07', 'P.'),
       ('unclassified_prom', 'P.'), ('Total', 'P.')])

In [131]:
def format_bases(num: int, factor: int = 1000, suffix: str = "B") -> str:
    """Return a human readable string of a large number using SI unit prefixes

    :pararm num: A number to convert to decimal form
    :param factor: The SI factor, use 1000 for SI units and 1024 for binary multiples
    :param suffix: The suffix to place after the SI prefix, for example use B for SI units and iB for binary multiples
    :return: The input number formatted to two decimal places with the SI unit and suffix

    :Example:

    >>> format_bases(1_000)
    '1.00 kB'
    >>> format_bases(1_000_000)
    '1.00 MB'
    >>> format_bases(1_630_000)
    '1.63 MB'
    >>> format_bases(1_000_000_000)
    '1.00 GB'
    """
    if isinstance(num, float) or isinstance(num, int):
        for unit in ["", "k", "M", "G", "T", "P", "E", "Z"]:
            if abs(num) < factor:
                return f"{num:3.2f}"
            num /= factor
        return f"{num:3.2f}"
    else:
        return num


def format_fold(x: int) -> str:
    """
    Format the fold change
    """
    if not isinstance(x, float):
        return x
    # Check if the value is an integer
    if isinstance(x, (int, float)) and x == int(x):
        return f"{int(x)}x"
    # Otherwise, format to one decimal place
    elif isinstance(x, float):
        return f"{x:.1f}x"
    return x

In [179]:
t = t.fillna("")
t.columns.names = [None, "End Reason"]
idx = pd.IndexSlice
subset = idx[
    :,
    [(metric, end_reason) for end_reason in ("Seq.", "Unb.") for metric in ["Yield"]],
]
idx2 = pd.IndexSlice
subset2 = idx2[:, ("Fold Enrichment", "")]
idx3 = pd.IndexSlice
subset3 = idx3[
    :,
    [
        ("Yield", "Seq."),
        ("Yield", "Unb."),
        ("Mean", "Seq."),
        ("Mean", "Unb."),
        ("N50", "Seq."),
        ("N50", "Unb."),
        ("Median Off Target Coverage", ""),
        ("Median Target Coverage", ""),
        ("Fold Enrichment", ""),
    ],
]
s = (
    t.style.hide(axis=0, level=1, names=True)
    .format(precision=0, subset=subset3, thousands=",")
    .format(format_bases, subset=subset)
    .relabel_index(
        t.columns.get_level_values(0).map(
            lambda x: "Yield\n(Gb)" if str(x) == "Yield" else x
        ),
        axis=1,
        level=0,
    )
    .format_index(escape="latex", axis=0)
    .format_index("\\textbf{{{}}}", escape="latex", axis=1, level=[0, 1])
    .to_latex(
        position_float="centering",
        hrules=True,
        column_format="".join(repeat("X ", len(t.columns))),
        environment="table",
    )
).replace("tabular", "tabularx").replace("\centering", "\centering\n\scriptsize")

In [180]:
print(s)

\begin{table}
\centering
\scriptsize
\begin{tabularx}{X X X X X X X X X X X X}
\toprule
 & \multicolumn{2}{r}{\textbf{Yield}} & \multicolumn{2}{r}{\textbf{Mean}} & \multicolumn{2}{r}{\textbf{N50}} & \textbf{Sample} & \textbf{Panel} & \textbf{Gene number} & \textbf{Median Off Target Coverage} & \textbf{Median Target Coverage} & \textbf{Fold Enrichment} \\
End Reason & \textbf{Seq.} & \textbf{Unb.} & \textbf{Seq.} & \textbf{Unb.} & \textbf{Seq.} & \textbf{Unb.} & \textbf{} & \textbf{} & \textbf{} & \textbf{} & \textbf{} & \textbf{} \\
\midrule
barcode01 & 334.52 & 3.47 & 1,926 & 554 & 8,149 & 555 & GM12878 & TruSight 170 Tumour Panel & 170 & 1 & 12 & 12 \\
barcode02 & 1.24 & 4.84 & 4,203 & 551 & 7,191 & 552 & NB4 & TruSight RNA Fusion Panel & 508 & 1 & 15 & 15 \\
barcode03 & 1.25 & 3.84 & 5,065 & 556 & 6,858 & 556 & 22Rv1 & COSMIC & 717 & 1 & 12 & 12 \\
unclassified\_grid & 170.91 & 3.66 & 798 & 733 & 923 & 792 & * & * & * & 0 & 1 & * \\
Total & 2.99 & 15.80 & 2,998 & 599 &  &  &  &  &  

In [174]:
len(t.columns)

12

'X X X X X X X X X X '